In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## I am just a beginner so I appreciate any suggestions and comments!

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import math


In [ ]:
df = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
sub = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/sample_submission.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df = df.drop(columns="id")

In [ ]:
def label_encoder(df):
    lb = LabelEncoder()
    for c in df.columns:
        if df[c].dtype == 'object':
            df[c] = df[c].astype('category')
            df[c] = lb.fit_transform(df[c])
    
    return df

In [ ]:
df_encoded = label_encoder(df)
test_encoded = label_encoder(test)

In [ ]:
def to_add_feature(df):
    
    df["cont8_cont0"] = df["cont8"]*df["cont0"]
    df["cont8_cont13"] = df["cont8"]*df["cont13"]
    df["cont8_cont11"] = df["cont8"]*df["cont11"]
    df["cont0_cont13"] = df["cont0"]*df["cont13"]
    df["cont11_cont13"] = df["cont11"]*df["cont13"]
    df["cont0_cont11"] = df["cont0"]*df["cont11"]
    
    return df

In [ ]:
added_features_df = to_add_feature(df)
added_features_test =  to_add_feature(test)

In [ ]:
X = added_features_df.drop(columns="target")
y = added_features_df["target"]

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold
lgb = lgb.LGBMRegressor(random_state=42)
kfold = KFold(n_splits=5,random_state=42,shuffle=True)

In cross validate function, we can't use RMSE for the scoring metric so I created an original one

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
import math

def rmse_score(y_true, y_pred):
    """RMSE (Root Mean Square Error) """
    mse = mean_squared_error(y_true, y_pred)
    rmse = math.sqrt(mse)
    return rmse

score_funcs = {
        'rmse': make_scorer(rmse_score),
    }

scores = cross_validate(lgb, X, y, cv=kfold,scoring=score_funcs)
scores['test_rmse']
mean_rmse = scores['test_rmse'].mean()
print("RMSE:", scores['test_rmse'])
print('Mean_RMSE:', mean_rmse)

In [ ]:
model = lgb.fit(X,y)

In [ ]:
#Visualize feature importance
import lightgbm as lgb
lgb.plot_importance(model, figsize=(20, 30),importance_type="gain")
plt.show()

In [ ]:
test = added_features_test.drop(columns="id")
y_pred=model.predict(test)
submission = sub.copy()
submission['target'] = y_pred
submission.to_csv("submission.csv", index = False)
submission.head()